Adapted from https://www.tutorialspoint.com/webgl/webgl_sample_application.htm

In [ ]:
import feedWebGL2.feedback as fd
from ipywidgets import interact, interactive, fixed, interact_manual
import numpy as np
fd.widen_notebook()
np.set_printoptions(precision=4)

corners = 0.5 * np.array([
    [1, 1, 1],
    [1, -1, -1],
    [-1, -1, 1],
    [-1, 1, -1],
])
colors = corners + 0.5

def tetrahedron_triangles(corners):
    triangles = np.zeros([4, 3, 3], dtype=np.float)
    for i in range(4):
        triangles[i, :i] = corners[:i]
        triangles[i, i:] = corners[i+1:]
    return triangles
    
triangles = tetrahedron_triangles(corners)
tcolors = tetrahedron_triangles(colors)

# make faces "flat colored"
if 0:
    for i in range(4):
        for j in range(3):
            tcolors[i,j] = colors[i]
            
def matrix1(phi, i=0):
    result = np.eye(4)
    result[0,0] = np.cos(phi)
    result[i,i] = np.cos(phi)
    result[0,i] = np.sin(phi)
    result[i,0] = -np.sin(phi)
    return result


def matrix(phi=0.0, theta=0.0, xt=0, yt=0, zt=0):
    M1 = matrix1(phi, 1)
    #print(M1)
    M2 = matrix1(theta, 2)
    #print(M2)
    M12 = M1.dot(M2)
    Mt = np.eye(4)
    Mt[3,0] = xt
    Mt[3,1] = yt
    Mt[3,2] = zt
    return M12.dot(Mt)

M = matrix(1.0, 2.0, 0.1, -0.1, 0.2)
M

In [ ]:
vertices = triangles.ravel()

def rotate(phi=-0.5, theta=0.0, xt=0.0, yt=0.0, zt=0.0):
    M = matrix(phi * np.pi, theta * np.pi, xt, yt, zt)
    assert np.abs(np.linalg.det(M) - 1.0) < 0.0001
    feedback_program.change_uniform_vector("rotation_matrix", M.ravel())
    feedback_program.run()
    return M

vertex_shader = """#version 300 es
    uniform mat4 rotation_matrix;
    in vec3 coordinates;
    in vec3 vcolor;
    out vec3 output_vertex;
    out vec3 coord_color;

    void main() {
        coord_color = vcolor;
        gl_Position = vec4(coordinates, 1.0);
        gl_Position = rotation_matrix * gl_Position;
        gl_Position[3] = 1.0;
        output_vertex = gl_Position.xyz;
    }
"""

fragment_shader = """#version 300 es
    // For some reason it is required to specify precision, otherwise error.
    precision highp float;
    in vec3 coord_color;
    //out vec4 color;
    out vec4 fragmentColor;

    void main() {
        fragmentColor = vec4(coord_color, 1.0);
    }
"""

feedback_program = fd.FeedbackProgram(
    program = fd.Program(
        vertex_shader = vertex_shader,
        fragment_shader = fragment_shader,
        feedbacks = fd.Feedbacks(
            output_vertex = fd.Feedback(num_components=3),
        ),
    ),
    runner = fd.Runner(
        vertices_per_instance = 3 * len(triangles),
        run_type = "TRIANGLES",
        uniforms = fd.Uniforms(
            rotation_matrix = fd.Uniform(
                default_value = list(M.ravel()),
                vtype = "4fv",
                is_matrix = True,
            ),
        ),
        inputs = fd.Inputs(
            coordinates = fd.Input(
                num_components = 3,
                from_buffer = fd.BufferLocation(
                    name = "coordinates_buffer", # start at the beginning, don't skip any values...
                ),
            ),
            vcolor = fd.Input(
                num_components = 3,
                from_buffer = fd.BufferLocation(
                    name = "colors_buffer", # start at the beginning, don't skip any values...
                ),
            ),
        ),
    ),
    context = fd.Context(
        buffers = fd.Buffers(
            coordinates_buffer = fd.Buffer(
                array=list(vertices),
            ),
            colors_buffer = fd.Buffer(
                array=list(tcolors.ravel()),
            )
        ),
        width = 600,
        show = True,
    ),
)

# display the widget and debugging information
feedback_program.debugging_display()
#feedback_program

In [ ]:
#feedback_program.run()
interact(rotate, phi=(-1.0, 1.0), theta=(-1.0, 1.0), xt=(-1.0, 1.0), yt=(-1.0, 1.0), zt=(-1.0, 1.0))

In [ ]:
#move_corner(x=-0.1)

In [ ]:
A1 = np.array(feedback_program.get_feedback("output_vertex"))
A1

In [ ]:
A2 = np.array(feedback_program.get_feedback("output_vertex"))
A2

In [ ]:
A1 - A2

In [ ]:
colors

In [ ]:
np.abs(np.linalg.det(M))